In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator(verbose=False).build(number)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count(), number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(5000)
x_test, y_test = parallel_dataset_generation(1000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9938536781754878, 0.9944813917678293)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9958540066607762, 0.9968280145187217)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe(max_rows=10)

In [9]:
y

,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,SurnameName,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
0,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,NaN,...,Error,Name,NaN,NaN,String,EMail,PhoneNumber,Date,BiologicalSex,NaN
1,ItalianFiscalCode,Error,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,NaN,Municipality,...,Error,Name,NameSurname,NaN,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
2,NaN,NaN,Error,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Error,Municipality,...,CountryCode,Name,SurnameName,NaN,String,NaN,NaN,Date,BiologicalSex,NaN
3,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,Error,Region,Error,...,CountryCode,NaN,NameSurname,Surname,String,NaN,PhoneNumber,Date,BiologicalSex,Boolean
4,ItalianFiscalCode,NaN,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,SurnameName,Surname,NaN,EMail,PhoneNumber,Date,BiologicalSex,Error


In [10]:
model.predict_dataframe(X)

,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,SurnameName,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
0,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,NaN,...,String,Name,NaN,NaN,String,EMail,PhoneNumber,Date,BiologicalSex,NaN
1,ItalianFiscalCode,Error,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,NaN,Municipality,...,String,Name,NameSurname,NaN,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
2,NaN,NaN,Error,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Error,Municipality,...,String,Name,SurnameName,NaN,String,NaN,NaN,Date,BiologicalSex,NaN
3,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,Error,Region,Error,...,String,NaN,NameSurname,Surname,String,NaN,PhoneNumber,Date,BiologicalSex,Boolean
4,ItalianFiscalCode,NaN,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,Name,SurnameName,Surname,NaN,EMail,PhoneNumber,Date,BiologicalSex,Error


In [11]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('Error', 'Document'): 507,
         ('Error', 'SurnameName'): 596,
         ('ItalianVAT', 'Error'): 23,
         ('NameSurname', 'Error'): 196,
         ('SurnameName', 'Error'): 194,
         ('SurnameName', 'NameSurname'): 46,
         ('Error', 'NameSurname'): 677,
         ('NaN', 'CountryCode'): 88,
         ('Error', 'ItalianZIPCode'): 24,
         ('Float', 'Integer'): 245,
         ('BiologicalSex', 'String'): 35,
         ('Error', 'String'): 269,
         ('CountryCode', 'NaN'): 59,
         ('Integer', 'Float'): 167,
         ('PhoneNumber', 'String'): 3,
         ('Error', 'ItalianVAT'): 48,
         ('ProvinceCode', 'String'): 22,
         ('Float', 'Error'): 8,
         ('SurnameName', 'String'): 149,
         ('Error', 'Float'): 13,
         ('Error', 'NaN'): 9,
         ('Error', 'PhoneNumber'): 43,
         ('NameSurname', 'SurnameName'): 36,
         ('Error', 'CountryCode'): 33,
         ('Error', 'Municipality'): 3,
         ('Boolean', 'String'): 21,
   